# Ejercicio Módulo 2
**Inteligencia Artificial - CEIA - FIUBA**

**Damián Nicolás Smilovich**

En este ejercicio deben implementar un algoritmo de búsqueda que no sea **Búsqueda Primero en Anchura (BFS)** para resolver el problema de la Torre de Hanoi. La nota máxima dependerá del algoritmo implementado:

- **Búsqueda Primero en Profundidad**: nota máxima 6.
- **Búsqueda de Costo Uniforme**: nota máxima 6.
- **Búsqueda de Profundidad Limitada con Profundidad Iterativa**: nota máxima 7.
- **Búsqueda Voraz usando la heurística dada en el aula virtual**: nota máxima 8.
- **Búsqueda Voraz usando una heurística desarrollada por vos**: nota máxima 9.
- **Búsqueda A\* usando la heurística dada en el aula virtual**: nota máxima 9.
- **Búsqueda A\* usando una heurística desarrollada por vos**: nota máxima 10.

La función debe devolver la salida correspondiente a la solución encontrada o `None si no se encontró una solución.

Además, debe calcular métricas de rendimiento que, como mínimo, incluyan:

- `solution_found`: `True` si se encontró la solución, `False` en caso contrario.
- `nodes_explored`: cantidad de nodos explorados (entero).
- `states_visited`: cantidad de estados distintos visitados (entero).
- `nodes_in_frontier`: cantidad de nodos que quedaron en la frontera al finalizar la ejecución (entero).
- `max_depth`: máxima profundidad explorada (entero).
- `cost_total`: costo total para encontrar la solución (float).

In [74]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi

In [80]:
def search_algorithm(number_disks=5) -> (NodeHanoi, dict):

    list_disks = [i for i in range(5, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    max_allowable_depth = 100

    for depth_limit in range(0, max_allowable_depth + 1):
        nodes_to_check = [NodeHanoi(state=initial_state)]
        nodes_explored = 1
        # sin esto puede entrar en un loop infinito
        states_seen = {initial_state}
        # máxima profundidad alcanzada en esta iteración
        max_depth = 0

        while True:
            if (len(nodes_to_check) == 0):
                metrics = {
                    "solution_found": False,
                    "nodes_explored": nodes_explored,
                    "states_visited": len(states_seen),
                    "nodes_in_frontier": len(nodes_to_check),
                    "max_depth": max_depth,
                    "cost_total": None,
                    "solution_efficiency [%]": 0
                }
                break
    
            # elimina el último nodo de la lista
            last_node = nodes_to_check.pop()
            if last_node.depth >= depth_limit:
                continue
    
            # expande el nodo (en la primer iteración no chequeamos si last_node es solución, pero por diseño no lo es)
            frontier = last_node.expand(problem=problem)
            # y appendea los nodos frontera (append garantiza lifo)
            for node in frontier:
                # aunque el nodo se corresponda con un estado ya visitado, se cuenta como exploreado
                nodes_explored += 1
                # si el estado ya fue visitado, no se verifica si es solución
                if node.state in states_seen:
                    continue

                nodes_to_check.append(node)
                states_seen.add(node.state)
                max_depth = node.depth if node.depth > max_depth else max_depth
                
                # chequea si es la solución
                if problem.goal_test(node.state):
                    print(node.state)
                    metrics = {
                        "solution_found": True,
                        "nodes_explored": nodes_explored,
                        "states_visited": len(states_seen),
                        "nodes_in_frontier": len(nodes_to_check),
                        "max_depth": max_depth,
                        "cost_total": node.state.accumulated_cost,
                        "solution_efficiency [%]": 100 * (2**number_disks - 1) / node.state.accumulated_cost
                    }
                    return node, metrics

    return None, metrics

Se prueba la función:

In [81]:
solution, metrics = search_algorithm(number_disks=5)

HanoiState:  |  | 5 4 3 2 1


Veamos las métricas:

In [82]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 173
states_visited: 84
nodes_in_frontier: 22
max_depth: 45
cost_total: 45.0
efficiency [%]: 68.88888888888889


Veamos el camino de estados desde el principio a la solución:

In [78]:
for nodos in solution.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 | 2 | 3 1>
<Node HanoiState: 5 4 2 |  | 3 1>
<Node HanoiState: 5 4 2 | 1 | 3>
<Node HanoiState: 5 4 | 1 | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 | 4 | 3 1>
<Node HanoiState: 5 | 4 2 | 3 1>
<Node HanoiState: 5 | 4 2 1 | 3>
<Node HanoiState: 5 3 | 4 2 1 | >
<Node HanoiState: 5 3 | 4 2 | 1>
<Node HanoiState: 5 3 2 | 4 | 1>
<Node HanoiState: 5 3 2 | 4 1 | >
<Node HanoiState: 5 3 | 4 1 | 2>
<Node HanoiState: 5 3 | 4 | 2 1>
<Node HanoiState: 5 | 4 3 | 2 1>
<Node HanoiState: 5 | 4 3 1 | 2>
<Node HanoiState: 5 2 | 4 3 1 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState:  | 4 3 2 | 5 1

Y las acciones que el agente debería aplicar para llegar al objetivo:

In [79]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from